In [23]:
%load_ext autoreload
%autoreload 2
import warnings
import csv
import os
import nltk
import pickle
import sklearn
import string, re
import urllib
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from tqdm import tqdm
import pandas as pd
warnings.filterwarnings('ignore')
#execute the following of not downloaded already:
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from langdetect import detect_langs
from langdetect.lang_detect_exception import LangDetectException

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create lyric corpus

In [8]:
songs = pd.read_csv('/home/sumedhshah94/w266_project_topmusichits/data/interim/songs_charts_new_2000s.csv')
hit_songs = songs[songs.hit==1]
lyrics_corpus = []
for index, row in hit_songs.iterrows():
    if not row['lyrics']:
        continue
    else:
        lyrics_corpus.append(row['lyrics'])
        

## Lyrics Tokenization & Token Lemmatization

In [24]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
#english = list(set(nltk.corpus.words.words()))
tokenizer = RegexpTokenizer(r'\w+')

def clean_docs_lemma(lyrics_list):
    cleaned = []
    
    # Lyric tokenizataion
    for lyric in lyrics_list:
        clean_lyric = tokenizer.tokenize(lyric.lower())
        cleaned.append(clean_lyric) #lowercase words
        
    # Lemmatize lyrics
    for s,song in enumerate(cleaned):
        lemmatized_tokens = []
        for token in song:
            lemmatized_tokens.append(lemmatizer.lemmatize(token))
        cleaned[s] = lemmatized_tokens

        
    return cleaned



In [25]:
lemmed_lyrics = clean_docs_lemma(lyrics_corpus)

## Remove stop words and profanities

In [28]:
# Remove Stop Words and Profanities
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
new_stop_words = ['ooh','yeah','hey','whoa','woah', 'ohh', 'was', 'mmm', 'oooh','yah','yeh','mmm', 'hmm','deh','doh','jah','wa']
stop_words.extend(new_stop_words)

for s,song in tqdm(enumerate(lemmed_lyrics)):
    filtered_text = []    
    for token in song:
        if token not in stop_words:
            filtered_text.append(token)
    lemmed_lyrics[s] = filtered_text

1608it [00:03, 495.12it/s]


In [ ]:
## STILL NEED TO REMOVE PROFANITIES

## Dictionary creation and occurrence-based Filtering

In [30]:
# Dictionary Creation and Filtering
from gensim.corpora import Dictionary

dictionary = Dictionary(lemmed_lyrics)
dictionary.filter_extremes(no_below = 100, no_above = 0.8)
print('Number of unique tokens:', len(dictionary))

Number of unique tokens:  312


## Bag-of-Words and Index to Dictionary Conversion

In [31]:
# BOW and Index to Dictionary Conversion
from gensim.corpora import MmCorpus

gensim_corpus = [dictionary.doc2bow(song) for song in lemmed_lyrics]

#create index to dictionary
temp = dictionary[0]  # "loads" the dictionary
id2word = dictionary.id2token

## Set model parameters and execute model training

In [41]:
# Set Model Parameters
# training parameters
chunksize = 1608
passes = 2
#iterations = 400
num_topics = 5

/home/sumedhshah94/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
# Execute Training
from gensim.models import LdaModel
lda_model = LdaModel(
corpus=gensim_corpus,
id2word=id2word,
chunksize=chunksize,
alpha='auto',
eta='auto',
#iterations=iterations,
num_topics=num_topics,
passes=passes
)

/home/sumedhshah94/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Display & Visualize Results

In [43]:
#import pprint
lda_model.print_topics()

/home/sumedhshah94/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.037*"one" + 0.034*"know" + 0.022*"got" + 0.021*"time" + 0.019*"baby" + 0.018*"girl" + 0.015*"gonna" + 0.015*"like" + 0.015*"run" + 0.014*"go"'),
 (1,
  '0.035*"know" + 0.029*"like" + 0.029*"go" + 0.024*"got" + 0.024*"nigga" + 0.024*"bitch" + 0.022*"let" + 0.020*"back" + 0.019*"oh" + 0.018*"shit"'),
 (2,
  '0.070*"oh" + 0.046*"love" + 0.043*"like" + 0.024*"know" + 0.017*"got" + 0.017*"let" + 0.016*"na" + 0.016*"wanna" + 0.016*"baby" + 0.015*"way"'),
 (3,
  '0.036*"get" + 0.035*"got" + 0.028*"nigga" + 0.026*"like" + 0.016*"know" + 0.015*"bitch" + 0.014*"never" + 0.014*"let" + 0.013*"cause" + 0.013*"keep"'),
 (4,
  '0.036*"know" + 0.026*"baby" + 0.020*"uh" + 0.019*"take" + 0.016*"want" + 0.016*"give" + 0.016*"tell" + 0.015*"never" + 0.015*"right" + 0.015*"cause"')]

In [44]:
# OPTIONAL: calculate coherence score
from gensim.models.coherencemodel import CoherenceModel

coherencemodel = CoherenceModel(model=lda_model, texts=gensim_corpus, dictionary=dictionary, coherence='c_v')
print(coherencemodel.get_coherence())

/home/sumedhshah94/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


nan


In [51]:
# Visualize LDA model using pyLDAvis
#!pip install pyLDAvis
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()



gensimvis.prepare(lda_model, gensim_corpus, dictionary)
#pyLDAvis.sklearn.prepare(lda_model, gensim_corpus, dictionary)
#pyLDAvis.enable_notebook()
#pyLDAvis.display(vis_data)

/home/sumedhshah94/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValueError: Only callable can be used as callback